In [49]:
import requests
import datetime
import pandas as pd

#取得當天日期
today = datetime.date.today().strftime("%Y%m%d")
today2 = datetime.date.today().strftime("%Y/%m/%d")
today_y = today2.split('/')[0]
today_m = today2.split('/')[1]
today_d = today2.split('/')[2]
today3 = '/'.join([today_y, today_m.lstrip('0'), today_d.lstrip('0')])
#準備URL
url1 = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=%s&stockNo=2330' % today
url2 = 'http://www.taifex.com.tw/chinese/3/3_5.asp'
payload = {'download':'','hdn_gostartdate':today3,'hdn_goenddate':today3,'syear':today_y,'smonth':today_m,'sday':today_d,'eyear':today_y,'emonth':today_m,'eday':today_d,'datestart':today2,'dateend':today2}

#取得資料
err_count = 0
while err_count < 3:
    try:
        res1 = requests.get(url1)
        res2 = requests.post(url2, data=payload)
        #注意編碼
        res2.encoding = 'utf-8'
        break
    except:
        sleep(5)
        err_count += 1
        continue
if err_count == 3:
    print('連線失敗')

In [50]:
#準備TWSE資料
json = res1.json()
header = json['fields']
data = json['data']
#取最新一筆資料
df1 = pd.DataFrame.from_records(data, columns=header).sort_index(ascending=False).head(1).reset_index(drop=True)
#修正日期格式
df1.iloc[0]['日期'] = today3

#準備匯率資料
df2 = pd.read_html(res2.text)
df2 = df2[2]
df2.columns = df2.iloc[0]
df2 = df2.drop(0).reset_index(drop=True).sort_index(ascending=False).head(1)

#合併DF
df3 = pd.merge(df1, df2, on='日期')
df3

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,美元／新台幣,人民幣／新台幣,歐元／美元,美元／日幣,英鎊／美元,澳幣／美元,美元／港幣,美元／人民幣,美元／南非幣,紐幣／美元


In [51]:
#更新資料至
#讀取CSV
#df4 = pd.read_csv('../data/fp_demo3.csv')

#讀取SqLite
import pandas.io.sql as pd_sql
import sqlite3 as sql
conn = sql.connect("../data/twse.db")
df4 = pd.read_sql_query("select * from demo3;", conn)

#檢查日期是否重複
if not ((df4['日期'] == today3)).any():
    df4 = df4.append(df3, ignore_index=True)
    #df4.to_csv('../data/fp_demo3.csv', sep=',', encoding='utf-8-sig', index=False) #回存CSV
    df4.to_sql("demo3", conn, if_exists="replace", index=False) #回存SqLite
df4.sort_index(ascending=False).head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,美元／新台幣,人民幣／新台幣,歐元／美元,美元／日幣,英鎊／美元,澳幣／美元,美元／港幣,美元／人民幣,美元／南非幣,紐幣／美元
63,2017/12/8,"38,183,260","8,678,808,540",231.00,231.00,224.50,227.00,+0.50,"9,485",30.015,4.532775,1.17435,113.52,1.3502,0.75125,7.80785,6.62175,13.695,0.6843
62,2017/11/30,"112,262,770","25,539,072,296",226.50,230.50,226.00,226.00,-8.50,"21,352",30.01,4.536972,1.1859,112.295,1.34635,0.75735,7.8082,6.61455,13.637,0.6841
61,2017/11/29,"26,535,797","6,234,161,137",235.00,236.00,234.50,234.50,+0.50,"6,976",29.99,4.544505,1.1868,111.465,1.3419,0.75895,7.805,6.5992,13.6086,0.69135
60,2017/11/28,"47,834,178","11,198,457,400",234.50,235.00,233.00,234.00,-3.00,"15,262",30.001,4.544851,1.1904,111.3,1.33285,0.76,7.8033,6.6011,13.7511,0.6925
59,2017/11/27,"32,247,327","7,696,717,391",240.00,240.00,237.00,237.00,-7.00,"14,743",30.006,4.548369,1.19385,111.24,1.3334,0.76185,7.802,6.5971,14.0394,0.6881


In [52]:
#定義nomalize方法
def regData(x):
    #格式化數字
    for i, row in x.iterrows():
        x.loc[i, '成交股數'] = row['成交股數'].replace(',', '')
        x.loc[i, '成交筆數'] = row['成交筆數'].replace(',', '')
    #做normalization(第i天資料/i-1天資料)
    for i, row in X.iterrows():
        if i <= len(X)-2:
            x.loc[i, '成交股數'] = int(x.loc[i+1, '成交股數'])/int(x.loc[i, '成交股數'])
            x.loc[i, '成交筆數'] = int(x.loc[i+1, '成交筆數'])/int(x.loc[i, '成交筆數'])
            x.loc[i, '美元／新台幣'] = float(x.loc[i+1, '美元／新台幣'])/float(x.loc[i, '美元／新台幣'])
    return x
#取得必要欄位，以前40筆作為training data
X = df4[['成交股數', '成交筆數', '美元／新台幣']]
X = regData(X.sort_index(ascending=False))
X = X.sort_index(ascending=True)
X.columns = ['成交股數[Normalize]', '成交筆數[Normalize]', '美元／新台幣[Normalize]']
df4 = pd.concat([df4, X.shift()], axis=1)
df4.head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,美元／新台幣,...,美元／日幣,英鎊／美元,澳幣／美元,美元／港幣,美元／人民幣,美元／南非幣,紐幣／美元,成交股數[Normalize],成交筆數[Normalize],美元／新台幣[Normalize]
0,2017/9/1,"17,805,405","3,851,834,126",215.00,217.50,215.00,216.50,0.00,"6,956",30.156,...,110.1,1.29145,0.7936,7.82595,6.5781,12.9461,0.71595,NaN,NaN,NaN
1,2017/9/4,"15,053,185","3,273,164,145",216.50,218.00,215.50,217.50,+1.00,"8,635",30.101,...,109.46,1.2942,0.7956,7.82535,6.5311,12.966,0.71705,0.845428,1.24137,0.998176
2,2017/9/5,"34,514,275","7,503,488,603",218.00,218.00,216.50,218.00,+0.50,"7,065",30.07,...,109.455,1.29255,0.79635,7.82525,6.55625,12.9729,0.71775,2.29282,0.818182,0.99897
3,2017/9/6,"28,148,528","6,117,444,609",216.50,218.00,216.00,217.00,-1.00,"6,999",30.105,...,108.71,1.3042,0.7991,7.8246,6.53555,12.90365,0.72335,0.815562,0.990658,1.00116
4,2017/9/7,"19,198,540","4,170,321,680",218.00,218.50,216.00,217.00,0.00,"6,586",30.062,...,109.015,1.3046,0.80015,7.8196,6.5072,12.8172,0.719,0.682044,0.940992,0.998572


In [53]:
#開始執行預測
#準備預測資料(今日資料/昨日資料)
df5 = df4.sort_index(ascending=False).head(2).reset_index(drop=True)
pred1 = int(df5.loc[0, '成交股數'].replace(',', ''))/int(df5.loc[1, '成交股數'].replace(',', ''))
pred2 = int(df5.loc[0, '成交筆數'].replace(',', ''))/int(df5.loc[1, '成交筆數'].replace(',', ''))
pred3 = float(df5.loc[0, '美元／新台幣'])/float(df5.loc[1, '美元／新台幣'])

#使用訓練好的Model預測
import pickle
#with open('../data/lin_r_model.pickle', 'rb') as f: #LinearRegression
with open('../data/log_r_model.pickle', 'rb') as f: #LogisticRegression
    regression_model = pickle.load(f)
    pred_result = regression_model.predict([[pred1, pred2, pred3]])
    print('預測結果: %f' % pred_result)
df5

預測結果: 1.000000


,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,美元／新台幣,...,美元／日幣,英鎊／美元,澳幣／美元,美元／港幣,美元／人民幣,美元／南非幣,紐幣／美元,成交股數[Normalize],成交筆數[Normalize],美元／新台幣[Normalize]
0,2017/12/8,"38,183,260","8,678,808,540",231.00,231.00,224.50,227.00,+0.50,"9,485",30.015,...,113.52,1.3502,0.75125,7.80785,6.62175,13.695,0.6843,0.340124,0.444221,1.00017
1,2017/11/30,"112,262,770","25,539,072,296",226.50,230.50,226.00,226.00,-8.50,"21,352",30.01,...,112.295,1.34635,0.75735,7.8082,6.61455,13.637,0.6841,4.23062,3.06078,1.00067


In [54]:
#建立結果網頁
from datetime import timedelta

#計算明天日期
tomorrow = (datetime.date.today() + timedelta(days=1)).strftime("%Y/%m/%d")
tomorrow_y = tomorrow.split('/')[0]
tomorrow_m = tomorrow.split('/')[1]
tomorrow_d = tomorrow.split('/')[2]
today5 = '/'.join([tomorrow_y, tomorrow_m.lstrip('0'), tomorrow_d.lstrip('0')])
#建立預測欄位(非預測欄位填入空值)
#pred_col = [today5, '', '', '', '', '', float(pred_result), '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''] #LinearRegression
pred_col = [today5, '', '', '', '', '', '', int(pred_result), '', '', '', '', '', '', '', '', '', '', '', '', '', ''] #LogisticRegression
pred_col = pd.DataFrame(pred_col).T
pred_col.columns = list(df5.columns)

#將預測欄位加入更新過的DF
df6 = df4.append(pred_col, ignore_index=True)

#輸出結果HTML(避免資料過於龐大，只取前30筆作顯示)
#df6.sort_index(ascending=False).head(30).to_html('../data/lin_daily_result.html', index=False) #LinearRegression
df6.sort_index(ascending=False).head(30).to_html('../data/log_daily_result.html', index=False) #LogisticRegression